# Popunjavanje NA vrednosti na setovima

U ovoj svesci cemo napraviti funkcije koje ce popuniti NA vrednosti u train test i valid setu. Prvo smo podelili set na tri dela a tek onda sredjujemo NA vrednosti, kako bismo izbegli curenje podataka. Sustina je da smo test i valid setove popunili na identican nacin kao trening set

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
valid = pd.read_csv('validation.csv')

In [3]:
train.shape

(51088, 34)

In [4]:
test.shape

(10948, 34)

In [5]:
valid.shape

(10947, 34)

In [6]:
print(train.isnull().sum())

RefId                                    0
IsBadBuy                                 0
PurchDate                                0
Auction                                  0
VehYear                                  0
VehicleAge                               0
Make                                     0
Model                                    0
Trim                                  1644
SubModel                                 5
Color                                    5
Transmission                             5
WheelTypeID                           2196
WheelType                             2200
VehOdo                                   0
Nationality                              4
Size                                     4
TopThreeAmericanName                     4
MMRAcquisitionAuctionAveragePrice       11
MMRAcquisitionAuctionCleanPrice         11
MMRAcquisitionRetailAveragePrice        11
MMRAcquisitonRetailCleanPrice           11
MMRCurrentAuctionAveragePrice          220
MMRCurrentA

In [7]:
print(test.isnull().sum())

RefId                                    0
IsBadBuy                                 0
PurchDate                                0
Auction                                  0
VehYear                                  0
VehicleAge                               0
Make                                     0
Model                                    0
Trim                                   342
SubModel                                 1
Color                                    1
Transmission                             1
WheelTypeID                            498
WheelType                              499
VehOdo                                   0
Nationality                              0
Size                                     0
TopThreeAmericanName                     0
MMRAcquisitionAuctionAveragePrice        6
MMRAcquisitionAuctionCleanPrice          6
MMRAcquisitionRetailAveragePrice         6
MMRAcquisitonRetailCleanPrice            6
MMRCurrentAuctionAveragePrice           46
MMRCurrentA

In [8]:
print(valid.isnull().sum())

RefId                                    0
IsBadBuy                                 0
PurchDate                                0
Auction                                  0
VehYear                                  0
VehicleAge                               0
Make                                     0
Model                                    0
Trim                                   374
SubModel                                 2
Color                                    2
Transmission                             3
WheelTypeID                            475
WheelType                              475
VehOdo                                   0
Nationality                              1
Size                                     1
TopThreeAmericanName                     1
MMRAcquisitionAuctionAveragePrice        1
MMRAcquisitionAuctionCleanPrice          1
MMRAcquisitionRetailAveragePrice         1
MMRAcquisitonRetailCleanPrice            1
MMRCurrentAuctionAveragePrice           49
MMRCurrentA

TRIM predstavlja nivo opreme svakog vozila. U setu ima preko 100 nivoa, jer svaki proizvodjac ima svoje nazive. Ipak, moguce je te oznake generalizovati na nekoliko tipova - osnovi, mid range, sportski, luksuzni. Istrazivanjem svakog trima uspeli da odredimo sta svaka oznaka predstavlja. 

Trim ima veliki broj nedostajucih vrednosti. Srecom, kolona model i make su kompletne, tako da na osnovu modela mozemo da predpostavimo koji je nivo opreme nedostajuci za skoro sve instance. Radi smanjenja kompleksnosti, za 10 najcescih modela smo proverili koji trim je najcesci, pa smo tim modelima dodelili najcescu vrednost za trim. Na primer, modelu Forenza je falio veliki broj trimova. Forence dolaze sa tri kompleta opreme, svaki od kojih se moze kategorisati u basic.

Za retke modele nije moguce popuniti trim na ovaj nacin, pa smo njima dodelili basic.

U funkciji categorize_trim smo uneli veliki broj trimova iz seta i grupisali ih u 6 kategorija.



In [9]:
def fill_missing_trim(df, train_df):
    # Pronađi 10 najčešćih modela kojima fali 'Trim'
    missing_trim_counts = df[df['Trim'].isna()]['Model'].value_counts().head(10)

    # Prolazimo kroz tih 10 modela i popunjavamo NaN vrednosti
    for model in missing_trim_counts.index:
        # Pronađi najčešći 'Trim' za dati model u train setu
        most_common_trim = train_df.loc[train_df['Model'] == model, 'Trim'].mode()
        
        # Ako ne možemo naći najčešći Trim, postavi 'Bas'
        if not most_common_trim.empty:
            most_common_trim_value = most_common_trim[0]
        else:
            most_common_trim_value = 'Bas'

        # Popuni NaN vrednosti za 'Trim' u datom modelu sa najčešćim 'Trim' ili 'Bas'
        df.loc[(df['Model'] == model) & (df['Trim'].isna()), 'Trim'] = most_common_trim_value
    
    # Za sve ostale modele koji nisu u top 10, popuni 'Trim' sa 'Bas'
    df.loc[df['Trim'].isna(), 'Trim'] = 'Bas'
    
    return df

In [10]:
def categorize_trim(trim):
    # Definišemo kategorije na osnovu trimova
    basic_trims = {"W/T","Nor","i","Bas","Cla","1","ES", "L", "S", "SE", "LX", "EX", "LE", "GLS", "DX", "XE", "GL", "SXT", "CX", "150", "ZX3", "ZX4", "ZX5", "XL", "LXi", "ZXW", "GXE"}
    mid_range_trims = {"L30","SLE","SES","LS", "LT", "SEL", "XLT", "XLS", "SLT", "CXL", "Adv", "Cus", "SV6", "CE", "Tou", "CXS", "GS", "STX","2"}
    sport_trims = {"s","GT", "ST", "R/T", "GTS", "SS", "Spo", "GTP", "ZTS", "Z71", "ZX2", "SVT", "XRS", "FX4"}
    luxury_trims = {"Lim", "Lar", "Lux", "LTZ", "Den", "GLS", "GTC", "L10", "L20", "Pro", "Max", "OZ"}
    special_trims = {"Edg","3", "Edd", "Z24", "Har", "LL", "JLX", "JLS", "Hyb", "Ent", "Ral", "ZR2", "Spe", "Ove", "RS", "Hig", "3 R", "Cin", "Exe", "Val", "Sta", "VP", "Pre", "Spy", "SC2", "SC1", "Spy", "XR", "eC", "Out", "Ult", "Maz"}

    # Podela u kategorije
    if trim in basic_trims:
        return 'Basic'
    elif trim in mid_range_trims:
        return 'Mid-Range'
    elif trim in sport_trims:
        return 'Sport'
    elif trim in luxury_trims:
        return 'Luxury'
    elif trim in special_trims:
        return 'Special'
    else:
        return 'Unknown'

In [11]:
def analyze_bad_buy(df, first_column):
    
    cross_nation = pd.crosstab(df[first_column], df['IsBadBuy'])
    cross_nation_p = cross_nation.div(cross_nation.sum(1).astype(float), axis=0)  
    combined = pd.concat([cross_nation, cross_nation_p], axis=1, keys=['Count', 'Percentage']) 
    combined_sorted = combined.sort_values(by=('Percentage', 1), ascending=False)
    
    return combined_sorted

Ubedljivo najcesci tip menjaca je automatik. Iako bi bilo preciznije proveravati tip za svaki model, odlucili smo se za brzi i efikasniji pristup. 

In [12]:
def fill_missing_transmission(df):
    
    
   
    df['Transmission'].fillna('auto', inplace=True)
    
    return df

Tip tockova smo popunili prema najcescem tipu tockova za svaki model automobila

In [13]:
def fill_wheel_type_by_make(df, train):
    
    for model in train['Model'].unique():
        # Pronalazak najčešće vrednosti za 'WheelType' za dati model
        mode_values = train.loc[train['Model'] == model, 'WheelType'].mode()
        
        # Ako postoji najčešća vrednost, koristi je, inače koristi 'Alloy'
        if not mode_values.empty:
            najcesci_wheel_type = mode_values[0]
        else:
            najcesci_wheel_type = 'Alloy'
        
        # Kreiranje maske za nedostajuće vrednosti u 'WheelType' za dati model
        mask = (df['WheelType'].isna()) & (df['Model'] == model)
        
        # Popunjavanje nedostajućih vrednosti
        df.loc[mask, 'WheelType'] = najcesci_wheel_type
        df['WheelType'].fillna('Alloy', inplace=True)
    return df

In [14]:
def fill_size_by_make(df, train):
    
    for model in train['Model'].unique():
        # Pronalazak najčešće vrednosti za 'WheelType' za dati model
        mode_values = train.loc[train['Model'] == model, 'Size'].mode()
        
        # Ako postoji najčešća vrednost, koristi je, inače koristi 'Alloy'
        if not mode_values.empty:
            najcesci_size = mode_values[0]
        else:
            najcesci_size = 'MEDIUM'
        
        # Kreiranje maske za nedostajuće vrednosti u 'WheelType' za dati model
        mask = (df['Size'].isna()) & (df['Model'] == model)
        
        # Popunjavanje nedostajućih vrednosti
        df.loc[mask, 'Size'] = najcesci_size
        df['Size'].fillna('MEDIUM', inplace=True)
    return df

Nacionalnost smo popunili po najcescoj nacionalnosti za svakog proizvodjaca.

In [15]:
def fill_missing_nationality(df, train):
    # Prolazak kroz jedinstvene vrednosti u koloni 'Make'
    for make in train['Make'].unique():
        # Pronađi najčešću 'Nationality' za trenutni 'Make'
        most_common_nationality = train.loc[train['Make'] == make, 'Nationality'].mode()
        
        # Ako postoji najčešća nacionalnost, popuni NaN vrednosti sa njom
        if not most_common_nationality.empty:
            most_common_nationality_value = most_common_nationality[0]
            df.loc[(df['Make'] == make) & (df['Nationality'].isna()), 'Nationality'] = most_common_nationality_value
    df['Nationality'].fillna('AMERICAN', inplace=True)
    return df

In [16]:
def fill_missing_top_three_american_name(df,train):
    # Prolazak kroz jedinstvene vrednosti u koloni 'Make'
    for make in train['Make'].unique():
        # Pronađi najčešću vrednost u koloni 'TopThreeAmericanName' za trenutni 'Make'
        most_common_top_three_name = train.loc[train['Make'] == make, 'TopThreeAmericanName'].mode()
        
        # Ako postoji najčešća vrednost, popuni NaN vrednosti sa njom
        if not most_common_top_three_name.empty:
            most_common_top_three_name_value = most_common_top_three_name[0]
            df.loc[(df['Make'] == make) & (df['TopThreeAmericanName'].isna()), 'TopThreeAmericanName'] = most_common_top_three_name_value
    df['TopThreeAmericanName'].fillna('GM', inplace=True)
    return df

U prethodnim analizama smo videli da sve raspodele svih mmr vrednosti lice na normalnu, te cemo nedostajuce vrednosti popuniti medijanom 

In [17]:
def fill_missing_mmrs_with_median(df, train_df):
    
    columns_to_fill = [
        'MMRAcquisitionAuctionAveragePrice',
        'MMRAcquisitionAuctionCleanPrice',
        'MMRAcquisitionRetailAveragePrice',
        'MMRAcquisitonRetailCleanPrice',
        'MMRCurrentAuctionAveragePrice',
        'MMRCurrentAuctionCleanPrice',
        'MMRCurrentRetailAveragePrice',
        'MMRCurrentRetailCleanPrice'
    ]
    
    
    for column in columns_to_fill:
   
        filtered_train_df = train_df[(train_df[column] != 0) & (train_df[column] != 1)]
        
        # Izračunavanje medijana
        median_value = filtered_train_df[column].median()
        
        # Popunjavanje NaN vrednosti sa izračunatim medijanom
        df[column].fillna(median_value, inplace=True)
    
    return df

Boja nije znacajn atributm pa smo sve popunili plavom.

In [18]:
def fill_missing_color_with_blue(df):
    
    df['Color'].fillna('BLUE', inplace=True)
    return df

Sledece kolone smo resili da odmah izbacimo

In [19]:
def drop_unwanted_columns(df):
    # Definišemo kolone koje treba obrisati
    columns_to_drop = [
        'RefId', 'PurchDate', 'SubModel', 'WheelTypeID', 
        'PRIMEUNIT', 'AUCGUART', 'IsOnlineSale', 'VNZIP1', 'BYRNO'
    ]
    
    # Brisanje kolona iz DataFrame-a
    df.drop(columns=columns_to_drop, inplace=True)
    
    return df

In [20]:
def process_dataset(df, train_df):
    df = drop_unwanted_columns(df) #
    df = fill_wheel_type_by_make(df,train_df) #
    df = fill_missing_trim(df, train_df) #
    df = fill_missing_transmission(df) #
    df = fill_missing_nationality(df, train_df) #
    df = fill_missing_top_three_american_name(df,train_df) #
    df = fill_missing_mmrs_with_median(df, train_df) #
    df = fill_missing_color_with_blue(df) #
    df = fill_size_by_make(df,train_df)
    return df

In [21]:
train = process_dataset(train,train)

In [22]:
print(train.isnull().sum())

IsBadBuy                             0
Auction                              0
VehYear                              0
VehicleAge                           0
Make                                 0
Model                                0
Trim                                 0
Color                                0
Transmission                         0
WheelType                            0
VehOdo                               0
Nationality                          0
Size                                 0
TopThreeAmericanName                 0
MMRAcquisitionAuctionAveragePrice    0
MMRAcquisitionAuctionCleanPrice      0
MMRAcquisitionRetailAveragePrice     0
MMRAcquisitonRetailCleanPrice        0
MMRCurrentAuctionAveragePrice        0
MMRCurrentAuctionCleanPrice          0
MMRCurrentRetailAveragePrice         0
MMRCurrentRetailCleanPrice           0
VNST                                 0
VehBCost                             0
WarrantyCost                         0
dtype: int64


In [23]:
test = process_dataset(test,train)
valid = process_dataset(valid,train)

In [24]:
print(test.isna().sum())

IsBadBuy                             0
Auction                              0
VehYear                              0
VehicleAge                           0
Make                                 0
Model                                0
Trim                                 0
Color                                0
Transmission                         0
WheelType                            0
VehOdo                               0
Nationality                          0
Size                                 0
TopThreeAmericanName                 0
MMRAcquisitionAuctionAveragePrice    0
MMRAcquisitionAuctionCleanPrice      0
MMRAcquisitionRetailAveragePrice     0
MMRAcquisitonRetailCleanPrice        0
MMRCurrentAuctionAveragePrice        0
MMRCurrentAuctionCleanPrice          0
MMRCurrentRetailAveragePrice         0
MMRCurrentRetailCleanPrice           0
VNST                                 0
VehBCost                             0
WarrantyCost                         0
dtype: int64


In [25]:
print(valid.isnull().sum())

IsBadBuy                             0
Auction                              0
VehYear                              0
VehicleAge                           0
Make                                 0
Model                                0
Trim                                 0
Color                                0
Transmission                         0
WheelType                            0
VehOdo                               0
Nationality                          0
Size                                 0
TopThreeAmericanName                 0
MMRAcquisitionAuctionAveragePrice    0
MMRAcquisitionAuctionCleanPrice      0
MMRAcquisitionRetailAveragePrice     0
MMRAcquisitonRetailCleanPrice        0
MMRCurrentAuctionAveragePrice        0
MMRCurrentAuctionCleanPrice          0
MMRCurrentRetailAveragePrice         0
MMRCurrentRetailCleanPrice           0
VNST                                 0
VehBCost                             0
WarrantyCost                         0
dtype: int64


In [26]:
train.to_csv('train_bezNA.csv',index=False)
test.to_csv('test_bezNA.csv',index=False)
valid.to_csv('valid_bezNA.csv',index=False)